<a href="https://colab.research.google.com/github/mwolinsky/Datasets/blob/main/NLP_UdeSA_2025_1_a_Tokenizaci%C3%B3n%2C_BoW_y_clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Tokenización, bag-of-words y clasificación de textos
## Procesamiento del Lenguaje Natural
## Maestría en Cs. de Datos, Universidad de San Andrés
### 2025
### Profesores: Juan Manuel Pérez y Bruno Bianchi



In [ ]:
!pip install datasets transformers scikit-learn unidecode

# Tokenización

Vamos a tokenizar un texto en palabras. Para esto vamos a usar la librería `spacy`, que es una librería popular de NLP con múltiples modelos pre-entrenados para una variedad de tareas.

Primero, bajamos el modelo en español e inglés:

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

In [ ]:
import spacy

nlp = spacy.load("es_core_news_sm")

text = "Estamos cursando la materia Procesamiento del Lenguaje Natural, dictada por el Dr. Pérez y el Dr. Bianchi en la Universidad de San Andrés"

doc = nlp(text)


Un objeto de spacy es un "iterable" que contiene tokens e información por cada uno de estos tokens

In [ ]:

print(f"{'Token':<20} {'POS':<20} {'Is Alpha':<20} {'Lemma':<20} {'Is Stop':<20}")
print("-"*140)
for token in doc:

    print(f"{token.text:<20} {token.pos_:<20} {token.is_alpha:<20} {token.lemma_:<20} { token.is_stop:<20}")

Algunas definiciones rápidas:

- **Token**: Un token es una unidad mínima de un texto. Puede ser una palabra, un número, un signo de puntuación, etc.
- **POS**: Part of speech. La categoría gramatical de una palabra. Por ejemplo, "perro" es un sustantivo, "corre" es un verbo, "rápidamente" es un adverbio, etc.
- **Alpha**: ¿el token es alfanumérico?
- **Lema**: La forma base de una palabra. Por ejemplo, el lemma de "corriendo" es "correr". Usualmente es un verbo en infinito. (en desuso en NLP moderno)
- **Stopword**: Palabras que "no aportan información" (en un modelo bag-of-words...). Por ejemplo, "el", "la", "un", "una", etc. (en desuso en NLP moderno)

Las últimas tres columnas suelen ser útiles para preprocesar texto o reducir tamaño de vocabularios.


**Pregunta**: ¿qué tokens les llaman la atención? ¿hubieran separado distinto el texto?

## Cálculo de un vocabulario

Recordemos que, dado un corpus de texto, el vocabulario es el conjunto de palabras únicas que aparecen en el texto.

Vamos a calcular un vocabulario de un conjunto de oraciones sencillo.

In [ ]:
textos = [
    "bolsa de palabras",
    "palabras de bolsa",
    "bolsa es una palabra",
    "palabra es una bolsa",
    "palabra no es una bolsa",
    "bolsa es una bolsa",
    "bolsa es una bolsa y es una palabra",
    "bolsa bolsa bolsa palabra es es una una y",
]

def tokenize(text):
    """
    Tokeniza un texto
    """
    return [token.text for token in nlp(text)]

tokenize(textos[0])

In [ ]:
def calcular_vocabulario(textos):
    # Uso set que es un conjunto (en el sentido matemático de la palabra) de elementos
    vocabulario = set()

    for texto in textos:
        tokens = tokenize(texto)
        for token in tokens:
            vocabulario.add(token)

    # Lo convertimos a una lista primero ¿por qué?
    return list(vocabulario)

calcular_vocabulario(textos)

Si tenemos un texto, podemos convertirlo a una lista de identificadores de tokens (esto va a ser particularmente útil para modelos neuronales)

Para programar esto, usamos dos diccionarios:

- itos: "int-to-string". Un diccionario que mapea un número a una palabra
- stoi: "string-to-int". Un diccionario que mapea una palabra a un número


In [ ]:
itos = dict(enumerate(calcular_vocabulario(textos)))
# Diccionario por comprensión :-), magia ninja de python
stoi = {v: k for k, v in itos.items()}

itos, stoi

Ahora, hagamos una función que convierta un texto a una lista de identificadores de tokens

In [ ]:
def to_idx(texto, UNK_IDX=-1):
    """
    Convierte un texto a una lista de índices
    """
    tokens = tokenize(texto)

    ret = []

    for token in tokens:
        ret.append(
            stoi.get(token, UNK_IDX)
        )

    return ret


En los casos de que aparezca una palabra que no está en el vocabulario, la función debería devolver un token especial, usualmente conocido como token desconocido (o UNK)

Decimos que tenemos en ese caso una palabra OOV (out-of-vocabulary).

### Ejercicio

Correr la función en ejemplos de oraciones sencillas. Probar con palabras que no estén en el vocabulario.


In [ ]:
to_idx("bolsa bolsa bolsas")

## Modelo Bag-of-words

Dado un vocabulario de tamaño $V$, el modelo bag-of-words representa cada texto $T$ como la cantidad de veces que aparece cada palabra en el vocabulario en $T$, ignorando el orden de las palabras.

**Ejercicio**

Dado el vocabulario anterior, implementar la función bag_of_words que recibe un texto y devuelve un vector de tamaño `len(itos)` con la cantidad de veces que aparece cada palabra en el texto.

In [ ]:
def bag_of_words(texto):
    """
    TODO: Implementar la función bag of words
    """

    # Inicializamos un vector de ceros
    bow = [0] * len(itos)

    ### BEGIN SOLUTION
    pass
    ### END SOLUTION
    return bow

Probar con algunos ejemplos.

Afortunadamente, podemos usar `sklearn` y su módulo `feature_extraction.text` para hacer esto de manera más sencilla.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

CountVectorizer?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

textos = [
    "bolsa de palabras",
    "palabras de bolsa",
    "bolsa es una palabra",
    "palabra es una bolsa",
    "palabra no es una bolsa",
    "bolsa es una bolsa",
    "bolsa es una bolsa y es una palabra",
    "bolsa bolsa bolsa palabra es es una una y",
]

vect = CountVectorizer()

vect.fit(textos)

En `vect.vocabulary_` podemos encontrar el vocabulario construido por `CountVectorizer`

In [ ]:
vect.vocabulary_

`vect.transform` convierte un conjunto de textos en su representación matricial

In [ ]:
vect.transform(textos).todense()

Armemos un dataframe para que nos muestre qué significa cada columna de la matriz

In [ ]:
import pandas as pd

# Doy vuelta y ordeno el vocabulario (perdón por lo horrendo)
vocab = {v:k for k, v in vect.vocabulary_.items()}
columns = [vocab[k] for k in range(len(vocab))]

df = pd.DataFrame(
    vect.transform(textos).todense(),
    columns=columns
)

df["text"] = textos


df

Este modelo se llama de "unigramas" porque sólo considera apariciones de una única palabra. ¿Podemos extender esto?

## n-gramas

Una forma de agregar cierto orden  es mediante el uso de n-gramas.

In [ ]:

vect = CountVectorizer(ngram_range=(1, 2))

vect.fit(textos)

# Doy vuelta y ordeno el vocabulario (perdón por lo horrendo)
vocab = {v:k for k, v in vect.vocabulary_.items()}
columns = [vocab[k] for k in range(len(vocab))]

df = pd.DataFrame(
    vect.transform(textos).todense(),
    columns=columns
)

df["text"] = textos


pd.options.display.max_columns = 50

df

In [ ]:
df.shape

## Análisis de Sentimientos


Carguemos un dataset de Sentiment Analysis con la librería `datasets`

¿Qué es `huggingface/datasets`? Un inmenso repositorio de datasets, originalmente de NLP pero que ahora contiene datos de todo tipo. Ya no es necesario andar buscando csvs ni jsons por lugares oscuros de Internet: con la api de `datasets` ya estamos.

Pueden ver qué tiene en https://huggingface.co/datasets, así como también su documentación

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "sentiment")

dataset

Un `Dataset` es un objeto que contiene un conjunto de datos con una estructura similar a un dataframe, aunque algo más restringido. Esto se debe principalmente a que está optimizado para usar grandes volúmenes de datos mediante el tipo de datos `parquet`.



In [ ]:
dataset["train"][1]

¿Qué son los valores de "label"?

In [ ]:
dataset["train"].features

0 es negativo, 1 neutral, y 2 positivo.

Usemos la función [`.map`](https://huggingface.co/docs/datasets/process#map) de datasets que nos ayuda a aplicar un procesado sobre los datos.

In [ ]:
label_map = dataset["train"].features["label"].names

label_map

`.map` toma una función, y nos pasa uno a uno cada elemento de nuestro dataset. Si devolvemos un diccionario con columnas `c1, ... cn`, nos devuelve otro dataset con los valores de dichas columnas que devolvimos para cada elemento del dataset.

(es parecido al `.apply` de pandas)

In [ ]:
# Convirtamos todo a strings...
# Cuando usemos pytorch no deberíamos hacer esto :-)
dataset = dataset.map(lambda ex: {"sentiment": label_map[ex["label"]] })

A ver cómo quedó...

In [ ]:
dataset

In [ ]:
# Para acceder a un "ejemplo"/elemento, hay que pedir el split y luego mandar el índice
dataset["train"][0]

In [ ]:
dataset["train"][120]

In [ ]:
dataset["train"][1001]

## Preprocesamiento

Hagamos un preprocesamiento muy básico: convertimos todo a minúsculas y convertimos repeticiones de 3 letras

In [ ]:
import re

re.sub(r"(.)\1{3,}", r"\1\1\1", "aaaaaaahhhhh no te la puedo CREEER")

Para preprocesar, usamos de nuevo `.map`

In [ ]:
import re

def preprocess_tweet(example):
    text = example["text"].lower()
    text = re.sub(r"(.)\1{3,}", r"\1\1", text)
    return {"text": text}

preprocessed_dataset = dataset.map(preprocess_tweet)

In [ ]:
preprocessed_dataset["train"][0]

Convirtamos a pandas y la seguimos ahí. Promo sólo válida para esta notebook.

In [ ]:

train_df = preprocessed_dataset["train"].to_pandas()
dev_df = preprocessed_dataset["validation"].to_pandas()
test_df = preprocessed_dataset["test"].to_pandas()

## A clasificar se ha dicho!

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(
    max_features=30_000,
    ngram_range=(1, 2),
)

classifier = LogisticRegression()

pipe = Pipeline([
    ("vectorizer", vectorizer),
    ("classifier", classifier)
])



pipe.fit(train_df["text"], train_df["sentiment"])


In [ ]:
from sklearn.metrics import classification_report

y_pred = pipe.predict(dev_df["text"])
y_true = dev_df["sentiment"]

print(classification_report(y_true, y_pred))

Veamos algunos errores...

In [ ]:
dev_df["pred"] = y_pred

In [ ]:
dev_df.loc[y_true != y_pred].sample(10, random_state=42)

## Ejercicio 1

Mejorar el preprocesado del modelo anterior

* Usemos un tokenizador especial para Tweets
* Reomover hashtags
* Sacar palabras muy frecuentes
* Convertir números a un token especial

Usar `nltk.tokenize.TweetTokenizer`

In [ ]:
from nltk.tokenize import TweetTokenizer

TweetTokenizer?

## Ejercicio 2

Sacar palabras muy frecuentes y muy infrecuentes del vectorizador. Usar para ello `max_df` y `min_df`

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vectorizer = CountVectorizer(
    max_features=30_000,
    ngram_range=(1, 2),
    #Todo: completar argumentos faltantes
    tokenizer=TweetTokenizer(reduce_len=True, strip_handles=True).tokenize
)

pipe = Pipeline([
    ("vectorizer", vectorizer),
    ("classifier", LogisticRegression())
])

pipe.fit(train_df["text"], train_df["label"])


y_pred = pipe.predict(dev_df["text"])
y_true = dev_df["label"]

print(classification_report(y_true, y_pred))

## Ejercicio 3

¿Para qué sirve el parámetro `binary` de `CountVectorizer`?

Probar si la utilización de ese parámetro mejora la performance


## Ejercicio 4

Evaluar en test los distintos algoritmos de clasificación



## Ejercicio 4

Ver la [tarea CoLA](https://huggingface.co/datasets/glue/viewer/cola/train) del benchmark GLUE. Describir brevemente qué tipo de tarea es y qué busca hacer.

¿Por qué funcionan mal los clasificadores propuestos en esta tarea?


In [ ]:
from datasets import load_dataset

ds = load_dataset("glue", "cola")

ds

## Ejercicio 5

a. Dado un vocabulario de tamaño $V$, ¿cuántos posibles n-gramas de tamaño $n$ podemos formar?

b. ¿Qué pasa cuando un n-grama aparece en el conjunto de entrenamiento pero no en el de test? ¿y al revés?


## Ejercicio 6.

Describa situaciones en las que el modelo bag-of-words puede fallar. ¿Qué tipo de información se pierde al usar este modelo?